# Install detectron2

In [ ]:
# install dependencies:
# (using +cu100 because colab is on CUDA 10.0)

!pip install -U torch==1.4+cu100 torchvision==0.5+cu100 -f https://download.pytorch.org/whl/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

import torch, torchvision
torch.__version__
!gcc --version

# Install Detectron2
!git clone https://github.com/facebookresearch/detectron2 detectron2_repo
pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu100/index.html
!cp detectron2_repo/configs ./ -r
# opencv is pre-installed on colab

Restart runtime in case you are installing detectron for the first time

In [13]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

import numpy as np
import cv2
import random
import os
import wandb

try:
    from google.colab.patches import cv2_imshow
except:
    os.system(f"""pip install google.colab""")
    from google.colab.patches import cv2_imshow
import glob
import matplotlib.pyplot as plt

# import some common .detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.structures import BoxMode

### Folder structure to follow to directly use this notebook

```
EndoCV_Det
│   class_list_bbox.txt  
│
└─── TRAIN
│      └ images
│      └ bbox
└─── VAL
│      └ images
│      └ bbox
└─── HOLDOUT  (Holdout set is optional)
       └ images
       └ bbox
```

In [2]:
def get_bb(img_path, img_name):
    """
        This function returns annotations in the format
        xyxy_ABS: xmin, ymin, xmax, ymax - ABSOLUTE
        
        Function built taking into consideration the dataset
        provided by EAD2020
        
        change the path to the bounding box folder(if not training on EAD)
        and change the loading method(if using .mat) accordingly
    """
    bb_path = img_path[:-7]+'bbox/'
    img = plt.imread(img_path+img_name)
    m, n, _ = img.shape
    labels = np.loadtxt(bb_path+img_name[:-4]+'.txt').reshape(-1, 5)
    classes = np.empty(len(labels), dtype=np.int32)
    xyxy = np.empty((len(labels), 4), dtype=np.int32)
    for i, label in enumerate(labels):
        cls, x, y, w, h = label
        x1 = (x-w/2.)
        x2 = x1 + w
        y1 = (y-h/2.)
        y2 = y1 + h
        x1 = np.clip(int(x1 * n), 0, n-1) ; x2 = np.clip(int(x2*n), 0, n-1)
        y1 = np.clip(int(y1 * m), 0, m-1) ; y2 = np.clip(int(y2*m), 0, m-1)
        classes[i] = int(cls)
        xyxy[i] = [x1, y1, x2, y2]
        
    return xyxy, classes


def _get_dicts(phase):
    if phase == 'train':
        path = 'EndoCV_Det/TRAIN/images/'
    elif phase == 'val':
        path = 'EndoCV_Det/VAL/images/'
    else:
        raise(Exception('Provide either "Train" or "Val"'))
    
    def get_dicts():
        dataset_dicts = []
        img_list = os.listdir(path)
        for idx, i in enumerate(img_list):
            record = {}
            img = plt.imread(path+i)
            height, width, _ = img.shape        
            record["file_name"] = path+i
            record["image_id"] = idx
            record["height"] = height
            record["width"] = width
            proposal_bb, proposal_logits = get_bb(path, i)
            objs=[]
            for j in range(len(proposal_bb)):
                obj = {
                    "bbox": [proposal_bb[j][0], proposal_bb[j][1], proposal_bb[j][2], proposal_bb[j][3]],
                    "bbox_mode": BoxMode.XYXY_ABS,
                    "category_id": proposal_logits[j],
                    "iscrowd": 0
                }
                objs.append(obj)
            record["annotations"] = objs
            record["thing_classes"] = ["specularity","saturation",
                                          "artifact", "blur", "contrast", "bubbles",
                                          "instrument", "blood"]
            dataset_dicts.append(record)
        return dataset_dicts
    return get_dicts

from detectron2.data import DatasetCatalog, MetadataCatalog
for d in ["train", "val"]:
    DatasetCatalog.register("endo1_" + d, _get_dicts(d))
    MetadataCatalog.get("endo1_" + d).set(thing_classes=["specularity","saturation",
                                      "artifact", "blur", "contrast", "bubbles",
                                      "instrument", "blood"])
endo_metadata = MetadataCatalog.get("endo1_train") 

## Visualisation

In [3]:
dataset_dicts = _get_dicts('train')()
for d in random.sample(dataset_dicts, 3):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=endo_metadata, scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    cv2_imshow(vis.get_image()[:, :, ::-1])
    

### To see available models:
Look under the folder `detectron2_repo/configs/COCO-Detection/` for config files available for detection models.


For eg:
    "detectron2_repo/configs/COCO-Detection/retinanet_R_50_FPN_3x.yaml"

# Train

In [4]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg

cfg = get_cfg()

config_file = "COCO-Detection/faster_rcnn_R_101_C4_3x.yaml"
# config_file = "COCO-Detection/faster_rcnn_R_101_DC5_3x.yaml"
# config_file = "COCO-Detection/retinanet_R_50_FPN_3x.yaml"
# config_file = "COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"

cfg.merge_from_file(f"configs/{config_file}")
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(config_file)

cfg.DATASETS.TRAIN = ("endo1_train",)
cfg.DATASETS.TEST = ("endo1_val",)
cfg.DATALOADER.NUM_WORKERS = 2

cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256 
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 8
cfg.MODEL.RETINANET.NUM_CLASSES = 8

In [ ]:
# Automatically switch to a new tensorboard directory every time this cell is executed
i = 1
name_desc = config_file  # Extended Name for the Run
notes= config_file + ": Serious run"  # Notes, if any

while os.path.exists(os.path.join(cfg.OUTPUT_DIR, f'run{i}')):
    i += 1

cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 5000
cfg.OUTPUT_DIR = os.path.join('./output/', f'run{i}')
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

# Create trainer
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)

# WandB
config = dict(cfg)
del config['MODEL']
wandb.init(
    name=f"Run {i}{': ' if name_desc else ''}{name_desc}",  # Name to display in wandb
    notes=notes,  # Notes, if any
    project='endocv-det',  # Project name as on WandB website
    sync_tensorboard=True,
    config=config  # Save hyperparameters
)

# Launch Tensorboard
%load_ext tensorboard
!pkill tensorboard
%tensorboard --logdir=output/run{i} --port=6007

# Commence Training
trainer.train()

## Inference & evaluation using the trained model



In [7]:
cfg.confidence_threshold = 0.4

from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")

trainer.resume_or_load(resume=False)

cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = cfg.confidence_threshold
cfg.MODEL.RETINANET.SCORE_THRESH_TEST = cfg.confidence_threshold
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = cfg.confidence_threshold
cfg.MODEL.PANOPTIC_FPN.COMBINE.INSTANCES_CONFIDENCE_THRESH = cfg.confidence_threshold 

In [ ]:
from detectron2.utils.visualizer import ColorMode

cfg.DATASETS.TEST = ("endo1_val", )
predictor = DefaultPredictor(cfg)

for t in ['train', 'val']:
    dataset_dicts = _get_dicts(t)()

    print("\n"*2 + t.upper() + "\n"*2)
    for d in random.sample(dataset_dicts, 10):
        im = cv2.imread(d["file_name"])
        outputs = predictor(im)
        v = Visualizer(im[:, :, ::-1],
                       metadata=MetadataCatalog.get(f"endo1_{t}"), 
                       scale=0.8, 
        )
        v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
        cv2_imshow(v.get_image()[:, :, ::-1])
        os.makedirs(os.path.join(cfg.OUTPUT_DIR, f'{t}_preds/'), exist_ok=True)
        
        # WandB
        wandb.log({f"{t}: {d['file_name']}": [wandb.Image(v.get_image(), caption=f"Threshold: {cfg.confidence_threshold}")]})

# Predict 
Predict on the test directory and create submission folder

In [14]:
def _get_dicts_test(folder='EndoCV_Det/TEST/images/'):
    path = folder
    def get_dicts_test():
        dataset_dicts = []
        img_list = os.listdir(path)
        for idx, i in enumerate(img_list):
            record = {}
            try:
                img = plt.imread(os.path.join(path, i))
                height, width, _ = img.shape
            except Exception as e:
                print(e)
                continue
            record["file_name"] = os.path.join(path, i)
            record["image_id"] = idx
            record["height"] = height
            record["width"] = width
            record["thing_classes"] = ["specularity","saturation",
                                          "artifact", "blur", "contrast", "bubbles",
                                          "instrument", "blood"]
            dataset_dicts.append(record)
        return dataset_dicts
    return get_dicts_test


def create_txt(a, name, folder, save_folder):
    """
       This function will create a .txt file of the name given
       or append to the existing one.
    """

    save_path = f"{save_folder}/{folder.lower() if folder != 'Detection_sequence' else 'sequence'}_bbox"
    os.makedirs(save_path, exist_ok=True)
    
    path=f"{save_path}/{name}"
    
    pred_boxes = a['pred_boxes'].tensor.detach().cpu().numpy()
    scores = a['scores'].detach().cpu().numpy()
    pred_classes = a['pred_classes'].detach().cpu().numpy()
    names = {0:'specularity', 1 : 'saturation', 2 : 'artifact', 3: 'blur',
         4 :'contrast', 5 : 'bubbles', 6 : 'instrument', 7 : 'blood'}
    
    f = open(path, 'a+')
    for i in range(len(pred_boxes)):
        x1, y1, x2, y2 = pred_boxes[i]
        label = pred_classes[i]
        confidence = scores[i]
        content = names[label]+" "+str(confidence)+" "+str(int((x1)))+" "+str(int(y1))+" "+str(int(x2))+" "+str(int(y2))+"\n"
        f.write(content)
    print(f'\rSaved {path}', end='')

In [ ]:
cfg.confidence_threshold = 0.4
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
visualize_preds = True
trainer.resume_or_load(resume=False)


from tqdm import tqdm_notebook as tqdm

cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = cfg.confidence_threshold
cfg.MODEL.RETINANET.SCORE_THRESH_TEST = cfg.confidence_threshold
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = cfg.confidence_threshold
cfg.MODEL.PANOPTIC_FPN.COMBINE.INSTANCES_CONFIDENCE_THRESH = cfg.confidence_threshold 

j = 1
while os.path.exists(f"EndoCV2020_testSubmission{j}"):
    j += 1
os.makedirs(f"EndoCV2020_testSubmission{j}")

for folder in ['Detection', 'Detection_sequence', 'Generalization']:
    print(f'\n\n{folder}\n\n')
    folder = os.path.join('EndoCV_Det/TEST/', folder)
    dataset_dicts = _get_dicts_test(folder=folder)()
    outputs = []
    files = []
    
    predictor = DefaultPredictor(cfg)
    
    print(f'Predicting on {folder}...', end='')
    for d in dataset_dicts:
        im = cv2.imread(d["file_name"])
        output = predictor(im)
        outputs.append(output)
        files.append(f"{os.path.basename(d['file_name'])[:-4]}.txt")
    print('Done!')
    
    if visualize_preds:
        print('Sample Predictions')
        for i in random.sample(range(len(outputs)), 3):
            output, file = outputs[i], files[i]
            im = cv2.imread(os.path.join(folder, file[:-4] + '.jpg'))
            v = Visualizer(im[:, :, ::-1],
                       metadata=MetadataCatalog.get(f"endo1_val"), 
                       scale=0.8, 
            )
            v = v.draw_instance_predictions(output["instances"].to("cpu"))
            cv2_imshow(v.get_image()[:, :, ::-1])
    
    
    for i in tqdm(range(len(outputs))):
        output = outputs[i]['instances'].get_fields()
        create_txt(output, files[i], os.path.basename(folder), f"EndoCV2020_testSubmission{j}")

## Visualize model's performance on any folder

In [15]:
def _get_dicts_test(folder='EndoCV_Det/HOLDOUT/images/'):
    path = folder
    def get_dicts_test():
        dataset_dicts = []
        img_list = os.listdir(path)
#         print(img_list)
        for idx, i in enumerate(img_list):
            record = {}
            try:
                img = plt.imread(path+i)
                height, width, _ = img.shape
            except:
                print(i)
                continue
            record["file_name"] = path+i
            record["image_id"] = idx
            record["height"] = height
            record["width"] = width
            record["thing_classes"] = ["specularity","saturation",
                                          "artifact", "blur", "contrast", "bubbles",
                                          "instrument", "blood"]
            dataset_dicts.append(record)
        return dataset_dicts
    return get_dicts_test

In [ ]:
folder = 'EndoCV_Det/TEST/Detection/'
cfg.confidence_threshold = 0.5
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
visualize_preds = True
n_samples = 10
trainer.resume_or_load(resume=False)

cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = cfg.confidence_threshold
cfg.MODEL.RETINANET.SCORE_THRESH_TEST = cfg.confidence_threshold
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = cfg.confidence_threshold
cfg.MODEL.PANOPTIC_FPN.COMBINE.INSTANCES_CONFIDENCE_THRESH = cfg.confidence_threshold 

from detectron2.utils.visualizer import ColorMode

cfg.DATASETS.TEST = ("endo1_val", )
predictor = DefaultPredictor(cfg)

dataset_dicts = _get_dicts_test(folder=folder)()

for d in random.sample(dataset_dicts, n_samples):
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1],
                   metadata=MetadataCatalog.get(f"endo1_val"), 
                   scale=0.8, 
    )
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(v.get_image()[:, :, ::-1])